# ETL Timeseries Import

This notebook illustrates how bulk import data from a users datasource.

## References 
* This notebook uses an example [dataset](https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+) from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php).
* The [Waylay api documentation](https://docs.waylay.io/api/)
* The [Waylay python SDK](https://docs.waylay.io/api/sdk/python/)
* [Setup instructions](https://github.com/waylayio/demo-general/tree/master/python-sdk) for a python notebook using the Waylay Python SDK.


## Parameters
Please review and adapt the following parameters for this demo

In [2]:
import tempfile

temp_dir_ctx = tempfile.TemporaryDirectory()

class ImportDemo:
    """parametrization for this demo"""
    
    # original location of the data set
    data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00357/occupancy_data.zip'
    
    # the profile name under which waylay credentials are stored
    waylay_client_profile='demo'
    
    # the id of the resource under which this demo is run
    resource_id = 'demo_occupancy_a'
    
    # a temporary directory that we can easily cleanup
    temp_dir = temp_dir_ctx.__enter__()

    @classmethod
    def cleanup(cls):
        temp_dir_ctx.__exit__(None,None,None)

## Setup

In [3]:
import pandas as pd
import waylay
from datetime import datetime

waylay.__version__

'v0.2.0'

## Data retrieval

### download the data set
We download the dataset (a zipped set of csv files), inspect its content, and read out the csv files into a pandas data structure.

In [4]:
import os
import os.path
import zipfile
from urllib.request import urlretrieve

os.makedirs('input', exist_ok=True)
os.makedirs('output', exist_ok=True)

# download the kaggle data set
if not os.path.isfile('input/occupancy.zip'):
    urlretrieve(ImportDemo.data_url, 'input/occupancy.zip')
    
with zipfile.ZipFile('input/occupancy.zip') as occ_zip:
    for file_name in occ_zip.namelist():
        print(file_name)

datatest.txt
datatest2.txt
datatraining.txt


In [5]:
with zipfile.ZipFile('input/occupancy.zip') as occ_zip:
    datatest = pd.read_csv(occ_zip.open('datatest.txt'))
    datatest2 = pd.read_csv(occ_zip.open('datatest2.txt'))
    datatraining = pd.read_csv(occ_zip.open('datatraining.txt'))
    


In [6]:
datatraining.describe()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
count,8143.000000,8143.000000,8143.000000,8143.000000,8143.000000,8143.000000
mean,20.619084,25.731507,119.519375,606.546243,0.003863,0.212330
std,1.016916,5.531211,194.755805,314.320877,0.000852,0.408982
min,19.000000,16.745000,0.000000,412.750000,0.002674,0.000000
25%,19.700000,20.200000,0.000000,439.000000,0.003078,0.000000
50%,20.390000,26.222500,0.000000,453.500000,0.003801,0.000000
75%,21.390000,30.533333,256.375000,638.833333,0.004352,0.000000
max,23.180000,39.117500,1546.333333,2028.500000,0.006476,1.000000


In [7]:
datatraining.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1
2,2015-02-04 17:51:59,23.15,27.2675,429.5,714.00,0.004783,1
3,2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1
4,2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1
5,2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1


## ETL Timeseries Bulk Import 
To bulk upload data into the Waylay Platform, the python supports SDK the following process.

<img src="ETL File Import.png">

This demo describes the general upload process, using Pandas dataframes.

Alternatively, you directly use CSV files, e.g. when files are too large to fit in memory.

The <a href="ETL import CSV conversion.ipynb">ETL import CSV conversion</a> notebook illustrates the different supported CSV formats.

The `etl_tool` service tool in the waylay client support this workflow.

In [8]:
waylay_client = waylay.WaylayClient.from_profile(ImportDemo.waylay_client_profile)
etl_tool = waylay_client.timeseries.etl_tool
etl_tool.temp_dir = ImportDemo.temp_dir

### convert to etl format
To upload bulk data into waylay, `etl_tool.prepare_import()` will convert your input source into a local CSV file that
has the right format.

In this case, we provide the tool with additional information:
 * `timestamp_timezone='UTC'` as timestamps do not contain a timezone component
 * `resource=ImportDemo.resource_id` as the resource id is not provided in the input
 * `timestamp_key='date'`, as timestamps are in the `date` column. The default if not specified is `timestamp`, but `date` will be recognised too.

The first two instruction are required for this dataset. Try to omit them to see what errors are raised.

In [10]:
etl_import = etl_tool.prepare_import(
    datatraining, 
    timestamp_timezone='UTC',
    resource=ImportDemo.resource_id,
    timestamp_column='date'
)
etl_import

100%|██████████| 6.00/6.00 [00:00<00:00, 16.1series/s]


WaylayETLSeriesImport(import_file=ETLFile(directory='/var/folders/07/zn347xhn33z8m79l8xtz1hn80000gp/T/tmpptr9hl_y', prefix='import-20210308.143130'), settings=SeriesSettings(metrics=['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy'], metric_column=None, metric=None, resources=['demo_occupancy_a'], resource_column=None, resource='demo_occupancy_a', value_column=None, timestamp_column='date', timestamp_offset=None, timestamp_first=None, timestamp_last=None, timestamp_interval=None, timestamp_constructor=None, timestamp_timezone='UTC'), storage_bucket='etl-import')

Because it is easer to work with recent data, we instruct the tool to shift timestamps
(with `timestamp_offset`, `timestamp_first` or `timestamp_last`)

In [11]:
etl_import = waylay_client.timeseries.etl_tool.prepare_import(
    datatraining, 
    timestamp_timezone='UTC',
    resource=ImportDemo.resource_id,
    timestamp_column='date',
    timestamp_last=datetime.utcnow(), # shift all timestamps so that last one is now
    temp_dir='output'
)
etl_import

100%|██████████| 6.00/6.00 [00:00<00:00, 15.5series/s]


WaylayETLSeriesImport(import_file=ETLFile(directory='output', prefix='import-20210308.143135'), settings=SeriesSettings(metrics=['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy'], metric_column=None, metric=None, resources=['demo_occupancy_a'], resource_column=None, resource='demo_occupancy_a', value_column=None, timestamp_column='date', timestamp_offset=None, timestamp_first=None, timestamp_last=datetime.datetime(2021, 3, 8, 14, 31, 35, 29123), timestamp_interval=None, timestamp_constructor=None, timestamp_timezone='UTC'), storage_bucket='etl-import')

The resulting file is a `gzip` compressed csv file in fully normalized _waylay timeseries ETL_ format

In [12]:
import gzip
with gzip.open(etl_import.import_file.path, 'rt') as csv_file:
     etl_series_df = pd.read_csv(csv_file)

etl_series_df.head()

,resource,metric,timestamp,value
0,demo_occupancy_a,Temperature,2021-03-02T22:49:35.029123Z,23.18
1,demo_occupancy_a,Temperature,2021-03-02T22:50:34.029123Z,23.15
2,demo_occupancy_a,Temperature,2021-03-02T22:51:35.029123Z,23.15
3,demo_occupancy_a,Temperature,2021-03-02T22:52:35.029123Z,23.15
4,demo_occupancy_a,Temperature,2021-03-02T22:53:35.029123Z,23.10


### (optional) create or update waylay resource
Timeseries in waylay are normally associated with a _Waylay Resource_, an entity that represents the device or source of your timeseries.

The `etl_tool` can extract resource metadata from your dataset, but you can enhance this information by providing addtional `Resource` and `Metric` descriptors.

In this case we take over the metadata descriptions that were available on the dataset.

In [13]:
from waylay.service.timeseries import Resource, Metric
hvac_resource_info = Resource(
    id= ImportDemo.resource_id,
    name= ImportDemo.resource_id,
    description =(
        "Experimental data used for binary classification (room occupancy) "
        "from Temperature,Humidity,Light and CO2.\n"
        "Ground-truth occupancy was obtained from time stamped pictures that were taken every minute.\n"
        "See https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+#"
    ),
    metrics = [
        Metric(name="Temperature", value_type="float", metric_type="gauge", unit="°C"), 
        Metric(name="Humidity", value_type= "float",  metric_type="gauge",  unit="%", description= "Relative Humidity"), 
        Metric(name="Light",value_type="float", metric_type="gauge",  unit= "Lux"), 
        Metric(name="CO2", value_type="float",  metric_type="gauge",  unit= "ppm"), 
        Metric(name="HumidityRatio", value_type="float", metric_type="gauge", unit="kgwater-vapor/kg-air", description="Derived quantity from temperature and relative humidity."),
        Metric(name="Occupancy", value_type="integer",  metric_type="gauge",  unit="boolean", description="0 for not occupied, 1 for occupied status")
    ]
)
hvac_resource_info.to_dict()

{'id': 'demo_occupancy_a',
 'description': 'Experimental data used for binary classification (room occupancy) from Temperature,Humidity,Light and CO2.\nGround-truth occupancy was obtained from time stamped pictures that were taken every minute.\nSee https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+#',
 'name': 'demo_occupancy_a',
 'metrics': [{'name': 'Temperature',
   'valueType': 'float',
   'metricType': 'gauge',
   'unit': '°C'},
  {'name': 'Humidity',
   'description': 'Relative Humidity',
   'valueType': 'float',
   'metricType': 'gauge',
   'unit': '%'},
  {'name': 'Light',
   'valueType': 'float',
   'metricType': 'gauge',
   'unit': 'Lux'},
  {'name': 'CO2', 'valueType': 'float', 'metricType': 'gauge', 'unit': 'ppm'},
  {'name': 'HumidityRatio',
   'description': 'Derived quantity from temperature and relative humidity.',
   'valueType': 'float',
   'metricType': 'gauge',
   'unit': 'kgwater-vapor/kg-air'},
  {'name': 'Occupancy',
   'description': '0 for not occupi

The following SDK calls upsert this resource into the Waylay system.

In [14]:
# use `update` (PATCH method) to upsert the resource
hvac_resource_resp = waylay_client.api.resource.update(ImportDemo.resource_id, body=hvac_resource_info.to_dict())

# validate it is stored correctly
waylay_client.api.resource.get(ImportDemo.resource_id)

{'id': 'demo_occupancy_a',
 'name': 'demo_occupancy_a',
 'metrics': [{'name': 'Temperature',
   'valueType': 'float',
   'metricType': 'gauge',
   'unit': '°C'},
  {'name': 'Humidity',
   'valueType': 'float',
   'metricType': 'gauge',
   'unit': '%',
   'description': 'Relative Humidity'},
  {'name': 'Light',
   'valueType': 'float',
   'metricType': 'gauge',
   'unit': 'Lux'},
  {'name': 'CO2', 'valueType': 'float', 'metricType': 'gauge', 'unit': 'ppm'},
  {'name': 'HumidityRatio',
   'valueType': 'float',
   'metricType': 'gauge',
   'unit': 'kgwater-vapor/kg-air',
   'description': 'Derived quantity from temperature and relative humidity.'},
  {'name': 'Occupancy',
   'valueType': 'integer',
   'metricType': 'gauge',
   'unit': 'boolean',
   'description': '0 for not occupied, 1 for occupied status'}],
 'description': 'Experimental data used for binary classification (room occupancy) from Temperature,Humidity,Light and CO2.\nGround-truth occupancy was obtained from time stamped pic

You will now be able to see the Resource in the [Waylay Console](https://preview.waylay.io/)

### upload the etl-import data

The next step will upload the import file to the `etl-import/upload` storage folder.

Any upload in this folder will initiate the following etl process:

* the file is moved from `etl-import/upload` to an timestamped folder in `etl-import/busy`
* the etl process is kicked of, reading data from this _busy_ folder.
* on completion, the file (and a result statement) is moved to a folder in `etl-import/done`

If anything goes wrong, either:
* the files are moved to an `ignored` folder if they do no comply with the requirements for an import.
* the files are moved to a `failed` folder if the etl process raised a fatal error
* note that, even if moved to the `done` folder, the processings results might still contain errors. Typically this is caused by parsing errors, e.g. if timestamps are not of the correct format.

The import files that are created by  _timeseries.etl_tool.prepare_import()_ should not run into `ignored` or parsing errors ...


The following reporting utilities allow you to follow up this process:
* `etl_tool.check_import(etl_import)` checks the status of a specific import job.
* `etl_tool.list_import(name_filter, status_filter)` queries the status of all jobs, optionally filtering on name and/or status.

In [15]:
etl_import = waylay_client.timeseries.etl_tool.initiate_import(etl_import)

Uploading content to etl-import/upload/import-20210308.143135-timeseries.csv.gz ...
... done.


In [16]:
from IPython.core.display import HTML, Markdown
resp = waylay_client.timeseries.etl_tool.check_import(etl_import)

HTML(resp.to_html())


In [17]:
for item in waylay_client.timeseries.etl_tool.list_import():
    display(HTML(item.to_html()))
    

### query the timeseries data

In [23]:
query = dict(
    resource=ImportDemo.resource_id,
    data=[
        dict(metric=metric) for metric in etl_import.settings.metrics
    ]
)
# test query
waylay_client.analytics.query.execute(
    body=query, 
    params=dict(until=datetime.utcnow().isoformat()
))

resource                         demo_occupancy_a                              \
metric                                Temperature Humidity  Light         CO2   
timestamp                                                                       
2021-03-07 05:13:35.029000+00:00            19.50  27.0000    0.0  456.000000   
2021-03-07 05:14:35.029000+00:00            19.50  27.0000    0.0  461.000000   
2021-03-07 05:15:34.029000+00:00            19.50  27.0000    0.0  458.000000   
2021-03-07 05:16:34.029000+00:00            19.50  27.0000    0.0  460.000000   
2021-03-07 05:17:35.029000+00:00            19.50  27.0000    0.0  462.000000   
...                                           ...      ...    ...         ...   
2021-03-08 14:27:35.029000+00:00            21.05  36.0975  433.0  787.250000   
2021-03-08 14:28:34.029000+00:00            21.05  35.9950  433.0  789.500000   
2021-03-08 14:29:34.029000+00:00            21.10  36.0950  433.0  798.500000   
2021-03-08 14:30:35.029000+00:00            21.10  36.2600  433.0  820.333333   
2021-03-08 14:31:35.029000+00:00            21.10  36.2000  447.0  821.000000   

resource                                                  
metric                           HumidityRatio Occupancy  
timestamp                                                 
2021-03-07 05:13:35.029000+00:00      0.003781       0.0  
2021-03-07 05:14:35.029000+00:00      0.003781       0.0  
2021-03-07 05:15:34.029000+00:00      0.003781       0.0  
2021-03-07 05:16:34.029000+00:00      0.003781       0.0  
2021-03-07 05:17:35.029000+00:00      0.003781       0.0  
...                                        ...       ...  
2021-03-08 14:27:35.029000+00:00      0.005579       1.0  
2021-03-08 14:28:34.029000+00:00      0.005563       1.0  
2021-03-08 14:29:34.029000+00:00      0.005596       1.0  
2021-03-08 14:30:35.029000+00:00      0.005621       1.0  
2021-03-08 14:31:35.029000+00:00      0.005612       1.0  

[1999 rows x 6 columns]

In [24]:
# save query
query_name = f'example_{ImportDemo.resource_id}'
waylay_client.analytics.query.create(body=dict(name=query_name, query=query))


{'data': [{'metric': 'Temperature'},
  {'metric': 'Humidity'},
  {'metric': 'Light'},
  {'metric': 'CO2'},
  {'metric': 'HumidityRatio'},
  {'metric': 'Occupancy'}],
 'resource': 'demo_occupancy_a'}

In [27]:
# test saved query
display(Markdown(
"Use the query in the console on\n"
f"* https://console.waylay.io/analytics/queries?query={query_name}"
))

waylay_client.analytics.query.data(query_name)

Use the query in the console on
* https://console.waylay.io/analytics/queries?query=example_demo_occupancy_a

resource                         demo_occupancy_a                              \
metric                                Temperature Humidity  Light         CO2   
timestamp                                                                       
2021-03-07 05:13:35.029000+00:00            19.50  27.0000    0.0  456.000000   
2021-03-07 05:14:35.029000+00:00            19.50  27.0000    0.0  461.000000   
2021-03-07 05:15:34.029000+00:00            19.50  27.0000    0.0  458.000000   
2021-03-07 05:16:34.029000+00:00            19.50  27.0000    0.0  460.000000   
2021-03-07 05:17:35.029000+00:00            19.50  27.0000    0.0  462.000000   
...                                           ...      ...    ...         ...   
2021-03-08 14:27:35.029000+00:00            21.05  36.0975  433.0  787.250000   
2021-03-08 14:28:34.029000+00:00            21.05  35.9950  433.0  789.500000   
2021-03-08 14:29:34.029000+00:00            21.10  36.0950  433.0  798.500000   
2021-03-08 14:30:35.029000+00:00            21.10  36.2600  433.0  820.333333   
2021-03-08 14:31:35.029000+00:00            21.10  36.2000  447.0  821.000000   

resource                                                  
metric                           HumidityRatio Occupancy  
timestamp                                                 
2021-03-07 05:13:35.029000+00:00      0.003781       0.0  
2021-03-07 05:14:35.029000+00:00      0.003781       0.0  
2021-03-07 05:15:34.029000+00:00      0.003781       0.0  
2021-03-07 05:16:34.029000+00:00      0.003781       0.0  
2021-03-07 05:17:35.029000+00:00      0.003781       0.0  
...                                        ...       ...  
2021-03-08 14:27:35.029000+00:00      0.005579       1.0  
2021-03-08 14:28:34.029000+00:00      0.005563       1.0  
2021-03-08 14:29:34.029000+00:00      0.005596       1.0  
2021-03-08 14:30:35.029000+00:00      0.005621       1.0  
2021-03-08 14:31:35.029000+00:00      0.005612       1.0  

[1999 rows x 6 columns]













##### cleanup

In [32]:
from waylay import RestResponseError
def cleanup(filter='demo_energy_hvac_occupancy', query_name_prefix='example_'):
    resource_ids = [ r['id'] for r in waylay_client.api.resource.search(params=dict(filter=filter)) ]
    if not resource_ids:
        print('No resources to clean.')
        return
    print('removing data and resources with ids:' + ''.join(f"\n  - {resource_id}" for resource_id in resource_ids))
    answer = input('OK? [Y/N] ')
        
    if not answer or answer[0].upper() != 'Y':
        print('Cleanup cancelled.')
        return
    
    # delete data
    for resource_id in resource_ids:
        try:
            print(waylay_client.api.resource.remove(resource_id) or f'removed resource {resource_id}')
            query_name = f'{query_name_prefix}{resource_id}'
            print(waylay_client.analytics.query.remove(query_name) or f'removed query {query_name}')
            # print(waylay_client.data.series.remove(resource_id)  or f'removed series   {resource_id}')
        except RestResponseError as exc:
            print(f'stopped processing resource {resource_id} because of:')
            print(exc)

In [ ]:
# cleanup()